# Stable Matching Pair Research
Implementation of [Stable Roomates](http://www.dcs.gla.ac.uk/~pat/jchoco/roommates/papers/Comp_sdarticle.pdf) for [Pair Research](http://pairresearch.io/). 

input from pair research: [(0, 1, 93), (0, 2, -20), (0, 3, 2), (1, 2, -13), (1, 3, 10), (2, 3, 80)]
- Single edge between two users with some weight for said edge

## Implementation of Standard Stable Roommates Matching 
What follows is a standard implmentation of the SR algorithm. It alone will **not** handle cases where (1) lists are not complete with preferences and (2) there are odd numbers of users.

In [1]:
from __future__ import print_function

import random
from copy import deepcopy


def stable_roommates(preferences, debug=False):
    """
    Runs complete algorithm returns a stable matching, if exists.

    Input:
        preferences (matrix, list of lists): n-by-m preference matrix containing preferences for each person.
            m = n - 1, so each person has rated all other people.
            Each row is a 1-indexed ordered ranking of others in the pool.
            Thus, max(preferences[person]) <= number people and min(preferences[person]) = 1.
        debug (boolean): including print statements

    Return:
        (list of tuples): stable matching, if exists. Otherwise, None.
    """
    # create a preference lookup table
    # person_number : [list of preferences]
    preferences_dict = {str(x + 1): [str(y) for y in preferences[x]] for x in range(len(preferences))}

    # create a dict of dicts holding index of each person ranked
    # person number : {person : rank_index }
    ranks = {index: dict(zip(value, range(len(value)))) for index, value in preferences_dict.iteritems()}

    # phase 1: initial proposal
    p1_holds = phase_1(preferences_dict, ranks)

    # if anyone does not have a hold, stable matching is not possible
    for hold in p1_holds:
        if p1_holds[hold] is None:
            if debug:
                print('Stable matching is not possible. Failed at Phase 1: not everyone was proposed to.')
            return None

    # phase 1: reduction
    p1_reduced_preferences = phase_1_reduce(preferences_dict, ranks, p1_holds)

    # phase 1: stable match halting condition
    # if p1_reduced_preferences has only one preference per person, matching should be stable (lemma 2)
    p1_halt = True
    for person in p1_reduced_preferences:
        if len(p1_reduced_preferences[person]) > 1:
            p1_halt = False
            break

    if p1_halt:
        # verification before returning
        if verify_matching(p1_holds):
            if verify_stability(p1_holds, ranks):
                if debug:
                    print('Stable matching found. Returning person : partner dictionary.')
                return p1_holds
            else:
                if debug:
                    print('Stable matching is not possible. Failed at Verification: matching computed, but not stable.')
                return None
        else:
            if debug:
                print('Stable matching is not possible. Failed at Verification: matching computed, but not valid.')
            return None

    # phase 2: find an all-or-nothing cycle
    cycle = find_all_or_nothing_cycle(p1_reduced_preferences, ranks, p1_holds)

    # if cycle with more than size 3 does not exist, no stable matching exists
    if cycle is None:
        if debug:
            print('Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle.')
        return None
    elif cycle is not None and len(cycle) == 3:
        if debug:
            print('Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle len > 3.')
        return None

    # phase 2: reduction
    final_holds = phase_2_reduce(p1_reduced_preferences, ranks, cycle)

    # check if holds are not empty
    if final_holds is not None:
        # verification
        if verify_matching(final_holds):
            if verify_stability(final_holds, ranks):
                if debug:
                    print('Stable matching found. Returning person : partner dictionary.')
                return final_holds
            else:
                if debug:
                    print('Stable matching is not possible. Failed at Verification: matching computed, but not stable.')
                return None
        else:
            if debug:
                print('Stable matching is not possible. Failed at Verification: matching computed, but not valid.')
            return None
    else:
        if debug:
            print('Stable matching is not possible. Failed at Phase 2 reduction.')
        return None


def phase_1(preferences, ranks, curr_holds=None):
    """
    Performs first phase of matching by doing round robin proposals until stopping condition (i) or (ii) is met:
         (i) each person is holding a proposal
        (ii) one person is rejected by everyone

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        curr_holds (dict, optional): dict of persons with current holds

    Return:
        (dict): holds after condition (i) or (ii) is met.
    """
    people = preferences.keys()

    # placeholder for holds
    holds = {person: None for person in people}

    # during phase 1, no holds exist. initialize curr_holds to 0 (all people are > 0)
    if curr_holds is None:
        curr_holds = {person: 0 for person in people}

    # randomize ordering of proposal
    random.shuffle(people)

    # track people who are already proposed to
    proposed_set = set()

    # begin proposing
    for person in people:
        proposer = person

        # proposal step
        while True:
            # find proposer someone to propose to, in order of proposer's preference list
            while curr_holds[proposer] < len(preferences[proposer]):
                # find proposee given proposer's preferences
                proposee = preferences[proposer][curr_holds[proposer]]
                curr_holds[proposer] += 1

                # find who proposee is holding, if any
                proposee_hold = holds[proposee]

                # stop searching if proposee doesn't hold anyone or ranks proposer higher than curr hold
                if proposee_hold is None or ranks[proposee][proposer] < ranks[proposee][proposee_hold]:
                    # proposee holds proposer's choice
                    holds[proposee] = proposer
                    break

            # check if proposee has already been proposed to
            if proposee not in proposed_set:
                # successful proposal
                proposed_set.add(proposee)
                break

            # if all preferences are exhausted and proposee does not have anyone, stop
            if curr_holds[proposer] >= len(preferences[proposer]):
                break

            # if proposee is proposed to, reject proposee_hold and continue proposal with them
            proposer = proposee_hold

    # final holds from phase 1
    return holds


def phase_1_reduce(preferences, ranks, holds):
    """
    Performs a reduction on preferences based on phase 1 proposals, and the following:
        Preference list for y who holds proposal x can be reduced by deleting
             (i) all those to whom y prefers x
            (ii) all those who hold a proposal from a person they prefer to y

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        holds (dict): dict of persons with current holds

    Return:
        (dict of list of strings): reduced preference list such that:
            (iii) y is the first on x's list and last on y's
             (iv) b appears on a's list iff a appears on b's
    """
    # create output preferences
    reduced_preferences = deepcopy(preferences)

    # loop though each hold
    for proposee in holds:
        proposer = holds[proposee]

        # loop though all of person's preferences
        i = 0
        while i < len(reduced_preferences[proposee]):
            # fetch proposee's preferences
            curr_proposee_preference = reduced_preferences[proposee][i]

            # proposee should only hold preferences equal and higher to proposer (i)
            if curr_proposee_preference == proposer:
                reduced_preferences[proposee] = reduced_preferences[proposee][:(i + 1)]
            # delete all people who hold a proposal from someone they prefer to the proposee (ii)
            elif ranks[curr_proposee_preference][holds[curr_proposee_preference]] < \
                    ranks[curr_proposee_preference][proposee]:
                reduced_preferences[proposee].pop(i)
                continue

            # continue to preference list
            i += 1

    return reduced_preferences


def find_all_or_nothing_cycle(preferences, ranks, holds):
    """
    Finds an all-or-nothing cycle in reduced preferences, if exists.

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        holds (dict): dict of persons with current holds

    Return:
        (list): cycle of users
    """
    # start with two individuals, p and q
    p = []
    q = []

    # find a person with > 1 preference left
    curr = None
    for person in preferences:
        if len(preferences[person]) > 1:
            curr = person
            break

    # if no person can be found, no cycle exists
    if curr is None:
        return None

    # create cycle
    while curr not in p:
        # q_i = second person in p_i's list
        q += [preferences[curr][1]]

        # p_{i + 1} = q_i's last person
        p += [curr]
        curr = preferences[q[-1]][-1]

    cycle = p[p.index(curr):]

    return cycle


def phase_2_reduce(preferences, ranks, cycle):
    """
    Performs a reduction on found all-or-nothing cycles.

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        cycle (list): all-or-nothing cycle

    Return:
        (dict): holds after sequential reductions, or None if no matching can be found.
    """
    # continue while a cycle exists
    curr_cycle = deepcopy(cycle)
    curr_holds = None
    p2_preferences = deepcopy(preferences)

    while curr_cycle is not None:
        curr_preferences = {}

        for person in preferences:
            if person in curr_cycle:
                curr_preferences[person] = 1
            else:
                curr_preferences[person] = 0

        curr_holds = phase_1(p2_preferences, ranks, curr_preferences)
        p2_preferences = phase_1_reduce(p2_preferences, ranks, curr_holds)

        curr_cycle = find_all_or_nothing_cycle(p2_preferences, ranks, curr_holds)

    return curr_holds


def verify_matching(matching):
    """
    Checks if a matching is valid.
        Valid matchings have all people matched to one and only one person.

    Input:
        matching (dict): dict containing person:matching pairs

    Return:
        (boolean)): matching is valid
    """
    # validate matching
    person_set = {person for person in matching.keys()}
    matching_set = {match for match in matching.values()}

    # equal cardinality and content
    if person_set != matching_set:
        return False

    # check for a:b, then b:a
    for person in matching:
        if person != matching[matching[person]]:
            return False

    # matching is valid
    return True


def verify_stability(matching, ranks):
    """
    Checks if a valid matching (all people matched to one and only one person) is stable.
        Stable iff no two unmatched members both prefer each other to their current partners in the matching.

    Input:
        matching (dict): dict containing person:matching pairs
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person

    Output:
        (boolean): matching is stable
    """
    for x in matching:
        for y in matching:
            # ignore if x, y are the same or x, y are matched
            if x == y or y == matching[x]:
                continue

            # get partner under matching for x, y and corresponding ranks of matched partners
            x_partner = matching[x]
            y_partner = matching[y]

            x_partner_rank = ranks[x][x_partner]
            y_partner_rank = ranks[y][y_partner]

            # get ranking of x -> y, y -> x
            x_y_rank = ranks[x][y]
            y_x_rank = ranks[y][x]

            # if x prefers y to current partner AND y prefers x to current partner, unstable
            # prefer = lower ranking index since ranking is highest -> lowest preference
            if x_y_rank < x_partner_rank and y_x_rank < y_partner_rank:
                return False

    return True


## Test Cases
A variety of test cases from (1) Irving's paper, (2) Wikipedia, (3) external implementations, and (4) any other custom cases.

### Test Case from Irving's Paper

In [2]:
paper_matching_6 = [
    [4, 6, 2, 5, 3],
    [6, 3, 5, 1, 4],
    [4, 5, 1, 6, 2],
    [2, 6, 5, 1, 3],
    [4, 2, 3, 6, 1],
    [5, 1, 4, 2, 3]
]

paper_matching_8 = [
    [2, 5, 4, 6, 7, 8, 3],
    [3, 6, 1, 7, 8, 5, 4],
    [4, 7, 2, 8, 5, 6, 1],
    [1, 8, 3, 5, 6, 7, 2],
    [6, 1, 8, 2, 3, 4, 7],
    [7, 2, 5, 3, 4, 1, 8],
    [8, 3, 6, 4, 1, 2, 5],
    [5, 4, 7, 1, 2, 3, 6]
]

paper_no_matching_4 = [
    [2, 3, 4],
    [3, 1, 4],
    [1, 2, 4],
    [1, 2, 3]
]

paper_no_matching_6 = [
    [2, 6, 4, 3, 5],
    [3, 5, 1, 6, 4],
    [1, 6, 2, 5, 4],
    [5, 2, 3, 6, 1],
    [6, 1, 3, 4, 2],
    [4, 2, 5, 1, 3]
]

In [3]:
stable_roommates(paper_matching_6, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '6', '2': '3', '3': '2', '4': '5', '5': '4', '6': '1'}

In [4]:
stable_roommates(paper_matching_8, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '4',
 '2': '3',
 '3': '2',
 '4': '1',
 '5': '6',
 '6': '5',
 '7': '8',
 '8': '7'}

In [5]:
stable_roommates(paper_no_matching_4, debug=True)

Stable matching is not possible. Failed at Phase 1: not everyone was proposed to.


In [6]:
stable_roommates(paper_no_matching_6, debug=True)

Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle len > 3.


### Test Cases from Wikipedia Article (https://en.wikipedia.org/wiki/Stable_roommates_problem#Algorithm)

In [7]:
wiki_matching_6 = [
    [3, 4, 2, 6, 5],
    [6, 5, 4, 1, 3],
    [2, 4, 5, 1, 6],
    [5, 2, 3, 6, 1],
    [3, 1, 2, 4, 6],
    [5, 1, 3, 4, 2]
]

In [8]:
stable_roommates(wiki_matching_6, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '6', '2': '4', '3': '5', '4': '2', '5': '3', '6': '1'}

### Test Cases from External Implementation (http://www.dcs.gla.ac.uk/~pat/roommates/distribution/data/) 

In [9]:
matching_8 = [
    [2, 5, 4, 6, 7, 8, 3],
    [3, 6, 1, 7, 8, 5, 4],
    [4, 7, 2, 8, 5, 6, 1],
    [1, 8, 3, 5, 6, 7, 2],
    [6, 1, 8, 2, 3, 4, 7],
    [7, 2, 5, 3, 4, 1, 8],
    [8, 3, 6, 4, 1, 2, 5],
    [5, 4, 7, 1, 2, 3, 6]
]

matching_10 = [
    [8, 2, 9, 3, 6, 4, 5, 7, 10],
    [4, 3, 8, 9, 5, 1, 10, 6, 7],
    [5, 6, 8, 2, 1, 7, 10, 4, 9],
    [10, 7, 9, 3, 1, 6, 2, 5, 8],
    [7, 4, 10, 8, 2, 6, 3, 1, 9],
    [2, 8, 7, 3, 4, 10, 1, 5, 9],
    [2, 1, 8, 3, 5, 10, 4, 6, 9],
    [10, 4, 2, 5, 6, 7, 1, 3, 9],
    [6, 7, 2, 5, 10, 3, 4, 8, 1],
    [3, 1, 6, 5, 2, 9, 8, 4, 7]
]

matching_20 = [
    [13, 12, 20, 17, 11, 6, 8, 2, 3, 14, 4, 16, 5, 10, 18, 19, 9, 15, 7],
    [13, 6, 8, 17, 18, 19, 1, 11, 7, 4, 15, 16, 5, 9, 3, 20, 12, 10, 14],
    [6, 16, 4, 9, 14, 13, 17, 19, 8, 2, 1, 12, 20, 5, 18, 15, 7, 11, 10],
    [11, 7, 8, 2, 17, 3, 15, 6, 19, 10, 9, 5, 1, 16, 13, 20, 18, 14, 12],
    [8, 17, 14, 16, 4, 13, 15, 6, 19, 9, 12, 7, 2, 3, 11, 18, 20, 10, 1],
    [8, 13, 10, 14, 18, 15, 2, 7, 4, 16, 19, 5, 9, 17, 20, 3, 11, 12, 1],
    [13, 1, 4, 9, 19, 18, 11, 14, 10, 2, 17, 6, 15, 16, 5, 3, 12, 8, 20],
    [1, 6, 20, 7, 5, 15, 19, 4, 12, 3, 17, 9, 10, 14, 16, 2, 18, 11, 13],
    [17, 13, 3, 5, 7, 4, 12, 2, 18, 20, 15, 8, 10, 1, 6, 11, 19, 14, 16],
    [9, 4, 16, 14, 18, 17, 15, 11, 20, 13, 3, 12, 2, 1, 19, 7, 5, 8, 6],
    [6, 15, 4, 1, 18, 14, 5, 3, 9, 2, 17, 13, 8, 7, 12, 20, 19, 10, 16],
    [5, 18, 7, 16, 6, 20, 19, 14, 9, 17, 3, 1, 8, 10, 11, 13, 2, 15, 4],
    [3, 10, 7, 18, 14, 15, 1, 6, 12, 4, 8, 19, 16, 17, 5, 20, 9, 11, 2],
    [2, 5, 10, 13, 19, 17, 6, 3, 18, 7, 20, 9, 1, 4, 16, 12, 15, 8, 11],
    [12, 13, 5, 11, 2, 16, 18, 14, 1, 6, 17, 8, 19, 4, 10, 7, 20, 3, 9],
    [1, 7, 6, 5, 14, 18, 12, 17, 20, 11, 15, 10, 2, 13, 3, 8, 19, 9, 4],
    [5, 8, 15, 9, 7, 18, 11, 10, 19, 2, 1, 12, 3, 14, 20, 13, 6, 16, 4],
    [14, 3, 8, 10, 13, 5, 9, 15, 12, 1, 17, 6, 16, 11, 2, 7, 4, 19, 20],
    [9, 15, 20, 12, 18, 1, 11, 5, 3, 2, 13, 14, 10, 7, 6, 16, 8, 17, 4],
    [5, 6, 18, 19, 16, 7, 4, 9, 2, 17, 8, 15, 1, 12, 13, 10, 14, 3, 11]
]

no_matching_7 = [
    [3, 4, 2, 6, 5, 7], 
    [6, 5, 4, 1, 3, 7], 
    [2, 4, 5, 1, 6, 7], 
    [5, 2, 3, 6, 1, 7],
    [3, 1, 2, 4, 6, 7],
    [5, 1, 3, 4, 2, 7],
    [1, 2, 3, 4, 5, 6]
]

In [10]:
stable_roommates(matching_8, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '4',
 '2': '3',
 '3': '2',
 '4': '1',
 '5': '6',
 '6': '5',
 '7': '8',
 '8': '7'}

In [11]:
stable_roommates(matching_10, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '4',
 '10': '8',
 '2': '9',
 '3': '6',
 '4': '1',
 '5': '7',
 '6': '3',
 '7': '5',
 '8': '10',
 '9': '2'}

In [12]:
stable_roommates(matching_20, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '8',
 '10': '14',
 '11': '15',
 '12': '16',
 '13': '7',
 '14': '10',
 '15': '11',
 '16': '12',
 '17': '5',
 '18': '9',
 '19': '20',
 '2': '6',
 '20': '19',
 '3': '4',
 '4': '3',
 '5': '17',
 '6': '2',
 '7': '13',
 '8': '1',
 '9': '18'}

In [13]:
stable_roommates(no_matching_7, debug=True) # odd case: add an additional person and rank them last in each person's list? 

Stable matching is not possible. Failed at Phase 1: not everyone was proposed to.


### Custom Test Cases

In [14]:
# odd filled with n + 1 row
custom_3 = [
    [3, 2, 4],
    [3, 1, 4],
    [1, 2, 4],
    [1, 2, 3]
]

In [15]:
stable_roommates(custom_3, debug=True)

Stable matching found. Returning person : partner dictionary.


{'1': '3', '2': '4', '3': '1', '4': '2'}

## old

In [16]:
class Person(object):
    """
    Person object with preferences. 
    
    Attributes:
        name: name of person, referred to by number
        preferences: ordered list of strings detailing pairing preferences. 
            Given n users in a pool, len(preferences) = n - 1 as each person ranks every other person in the pool.
        preference_index: dict containing index of preference for fast lookup of ranking.
    """
    def __init__(self, name, preferences):
        """
        Returns a Person object with attributes initialized.
        
        Input:
            preferences (list of strings): ordered list of strings detailed pairing preferences
                ex w/4 users: [[]]
        """
        self.name = str(name)
        self.preferences = preferences + [name]
        self.preference_index = {}
        
        # generate preference index
        for i in xrange(len(preferences)): 
            self.preference_index[preferences[i]] = str(i)
    
    def leftmost(self): 
        """
        Gets next choice for person to propose to.
        """
        return self.preferences[0]
    
    def rightmost(self): 
        """
        Gets next choice for person to propose to.
        """
        return self.preferences[0]           
    
class ResearchPool(object):
    """
    Pool of Persons to match over.
    
    Attributes:
        persons: list of people in pool
        person_index: dict containing name:Person mapping for fast lookup of people
        proposal_set: set of proposals made
    """
    def __init__(self, preference_matrix):
        """
        Returns a ResearchPool object with attributes intitialized.

        Input:
            preference_matrix (matrix, list of lists): n-by-m matrix of n people with m = n - 1 preferences each.
                preferences should be 1-indexed as each person will be 1-indexed.
        """
        self.persons = [Person(x + 1, preference_matrix[x]) for x in xrange(len(preference_matrix))]
        self.person_index = {}
        self.proposal_set = set()

        # check for an odd number of people
        if (len(self.persons) % 2) != 0: 
            self.persons.append(Person(len(self.persons), [x for x in xrange(len(self.persons[0]))]))
            
        # create person_index
        for i in self.persons: 
            self.person_index[i.name] = i
            
    def clear_proposal_set(self):
        """
        Empties the proposal_set attribute.
        """
        self.proposal_set = set()
    
    def get_person(self, person_name):
        """
        Get and return a Person in the research pool, by name
        
        Input: 
            person_name (string): name of person.
        
        Return: 
            (Person): Person object with the name
        """
        return self.persons[self.person_index[person_name]]

    def __str__(self):
        """
        Custom print for research pool in the format user | preference list.
        """
        output = ""
        for i in xrange(len(self.persons)):
            preference_string = " ".join([str(x) for x in self.persons[i].preferences])
            output += "{} | {}".format(i + 1, preference_string)
            output += "\n"
            
        return output

In [17]:
def create_pool(preference_matrix):
    """
    Creates a ResearchPool that stable matching can be used upon. 
    
    Input: 
        preference_matrix (matrix, list of lists): and n-by-m matrix of n people with m = n - 1 preferences each
    
    Return: 
        (ResearchPool): a ResearchPool object initialized with the preference_matrix
    """
    # validate preference matrix size
    n = len(preference_matrix)
    m = n - 1

    for i in preference_matrix:
        if len(i) is not m: 
            raise TypeError("Preference matrix must be n-by-m with m = n - 1 for all n.")
    
    return ResearchPool(preference_matrix)

pool = create_pool([[4, 6, 2, 5, 3],
                    [6, 3, 5, 1, 4],
                    [4, 5, 1, 6, 2],
                    [2, 6, 5, 1, 3],
                    [4, 2, 3, 6, 1],
                    [5, 1, 4, 2, 3]])

print(pool)

1 | 4 6 2 5 3 1
2 | 6 3 5 1 4 2
3 | 4 5 1 6 2 3
4 | 2 6 5 1 3 4
5 | 4 2 3 6 1 5
6 | 5 1 4 2 3 6

